In [30]:
import pandas as pd
import numpy as np
pd.set_option('display.max_columns', None)
runtime = pd.read_parquet("../raw-data/runtimeSeveralRoutesOctWithLags_all.gzip")

Form a table with:

- Rows representing the active trips at "2022-10-26 08:30:16".
- Columns containing information of:
  - Route
  - Direction
  - Current stop ID and stop name
  - Stop IDs and stop names 11-20 stops ahead

In [72]:
# Select 
query = "(routeId.isin(['21', '47', '33'])) & (serviceDate == '2022-10-26')"
runtime_sel = runtime.query(query).copy()

In [73]:
# All the trips on this day with their time range
trips = runtime_sel.groupby('tripId')["observedToStopArrivalTime"].agg(["min", "max"])

In [74]:
desired_time = "2022-10-26 08:30:16"

trips_sel = trips.query("min < @desired_time < max").copy()
tripIds_sel = trips_sel.index.tolist()

In [76]:
runtime_used = runtime_sel.query("tripId in @tripIds_sel").copy()


In [80]:
runtime_used["observedFromStopDepartureTime"] = runtime_used[
  "observedToStopArrivalTime"
] - pd.to_timedelta(
  runtime_used["observedRuntimeSeconds"], 
  unit='s'
)

In [81]:
# First, understand what the next 10-20 stops are for each instance
runtime_used = runtime_used[[
    "routeId", "directionId", "tripId", "toStopId",
    "toStopName", "toStopSequence",
    "observedFromStopDepartureTime", "observedToStopArrivalTime",
]]

In [82]:
sorted = runtime_used.sort_values(
    ["routeId", "directionId", "tripId", "toStopSequence"]
)

In [83]:
for step in range(10, 21):
  sorted[f"next_{step}_stopId"] = sorted.groupby([
    "routeId", "directionId", "tripId"
  ])["toStopId"].shift(-step)
  sorted[f"next_{step}_stopName"] = sorted.groupby([
    "routeId", "directionId", "tripId"
  ])["toStopName"].shift(-step)

In [85]:
# Now select the instances that fall within the desired timeframe
final = sorted.query(
  "observedFromStopDepartureTime < @desired_time < observedToStopArrivalTime"
).copy()